In [ ]:
import tkinter as tk
import tkinter.filedialog as fd
from selenium.webdriver.common.by import By
# root = tk.Tk()
# file = fd.askopenfilename(parent=root, title='Choose a record file')
# file2 = fd.askopenfilenames(parent=root, title='Choose a pdf files')
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from IPython.display import clear_output
# root.destroy()
from time import sleep
import easygui
import time
import math
import re
from datetime import date
from datetime import timedelta
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium import webdriver
import pandas as pd 
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import datetime
from dateutil.relativedelta import relativedelta

from selenium import webdriver
#set chromedriver.exe path
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.implicitly_wait(0)
#maximize browser
driver.maximize_window()
#launch URL
driver.get("https://www.reliancedigital.in/polycab-400-mm-elanza-pw01-wall-fan-white-grey/p/492664609")


In [ ]:
driver.implicitly_wait(2)
cc=['RAC (Fixed speed)','Direct cool refrigerator','Ceiling Fans']

list=[
'https://www.reliancedigital.in/air-conditioners/c/S101510?searchQuery=Air%20Conditioners%20Non%20Inverter:relevance',
'https://www.reliancedigital.in/search?q=Single%20Door%20Refrigerators:relevance',
'https://www.reliancedigital.in/search?q=Ceiling%20Fans:relevance'
]

for w in range(len(list)):
    category=str(cc[w])
    driver.get(str(list[w]))
    try:
        driver.find_element(by=By.XPATH, value='/html/body/div[5]/div[2]/div[3]/button[1]').click()
    except:
        pass
    #Get product link
    title=[]
    link=[]
    a=driver.find_element(by=By.CLASS_NAME, value='pl__headline').text
    a=a.split(' ')
    a=math.floor(int(a[-2])/24)
    for i in range(a+1):
        try:
            driver.get(str(list[w])+'&page='+str(i))
            time.sleep(1)
            job=driver.find_element(by=By.CLASS_NAME, value='pl__container')
            soup=BeautifulSoup(job.get_attribute('innerHTML'),'html.parser')
            a=soup.find_all("a",class_="")
            for job_elem in a:
                link.append('https://www.reliancedigital.in'+job_elem['href'])
            c=soup.find_all("p",class_="sp__name")
            for job_elem in c:
                title.append(job_elem.text)
        except:
            break
            
    df = pd.DataFrame({
    'Title': title,
    'Link': link})
    df.to_excel(str(category)+'.xlsx')

    #Get images link
    driver.implicitly_wait(0)
    About=[]
    prodDetails=[]
    specs=[]
    mainimg=[]
    for j in range(0,len(df)):
        img=[]
        print(j)
        driver.get(df['Link'][j])
        try:
            job=driver.find_element(by=By.CLASS_NAME, value='infiniteScroller')
            job=driver.find_element(by=By.CLASS_NAME, value='infiniteScroller')
            job=driver.find_element(by=By.CLASS_NAME, value='infiniteScroller')
            soup=BeautifulSoup(job.get_attribute('innerHTML'),'html.parser')
            a=soup.find("div",class_="blk__sm__7 flush__left pdp__topBlock")
            About.append(a.text)
        except:
            About.append('none')
        try:
            a=soup.find("div",class_="pdp__tab-info")
            prodDetails.append(a.text)
        except:
            prodDetails.append('none')
        try:
            a=soup.find("div",id="pdp__specification")
            specs.append(a.text)
        except:
            specs.append('none')
        try:
            driver.find_element(by=By.ID, value='RIl_PDPSlideShow').click()
            time.sleep(1)
            d=driver.find_elements(by=By.CLASS_NAME, value='mb__16')
            l=len(d)
            for i in range(l):
                try:
                    d[i].click()
                    job=driver.find_element(by=By.XPATH, value='/html/body/div[1]/main/div[2]/div/section[1]/div[1]/div[3]/div[2]/div/div/div/div[2]/div/div[2]/div[2]')
                    soup=BeautifulSoup(job.get_attribute('innerHTML'),'html.parser')
                    a=soup.find_all("img",class_="")
                    for item in a:
                        img.append('https://www.reliancedigital.in/'+item['src'])
                except:
                    pass
        except:
            img.append('none')
        mainimg.append(img)

    df['Points']=0
    df['image link']=0
    df['prodDetails']=0
    df['Specs']=0
    df['Points']=About
    df['Specs']=specs
    df['prodDetails']=prodDetails
    df['image link']=mainimg
    df.to_csv('final'+str(category)+'.csv')